pip install opencv-python
pip install pytesseract
pip install selenium
pip install typing_extensions

In [1]:
# pip install Preprocessing

In [2]:
# pip install webdriver-manager

In [1]:
%run Preprocessing.ipynb

C:\Users\MBBLABS\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\MBBLABS\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [3]:
import requests
#import cv2
import numpy as np

import pytesseract
import cv2
from matplotlib import pyplot as plt
import re
import requests
from bs4 import BeautifulSoup

C:\Users\MBBLABS\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\MBBLABS\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [6]:
import cv2
from bs4 import BeautifulSoup
import requests
import pytesseract
import warnings
warnings.filterwarnings("ignore")

In [4]:
# Get image from external file(path)
def get_image_path(path):
    """Load image from local directory.

    :param path: given path where image is stored
    :return: image loaded from directory
    """
    loaded_image = cv2.imread(path)
    loaded_image = cv2.cvtColor(loaded_image, cv2.COLOR_BGR2GRAY)
    label = ((path.split('/')[-1]).split('\\')[-1]).split('.')[0]
    return loaded_image, label

In [8]:
import cv2
loaded_image=cv2.imread("C:/Users/MBBLABS/Documents/Web Scrapping/captcha1")


In [5]:
# Download captcha images from link and save
def get_image_link(link):
    """Download image from given url.

    :param link: given url where image is located
    :return: image downloaded from url
    """
    req = requests.get(link)
    try:
        req.raise_for_status()
    except Exception as exc:
        print('There was a problem: %s' % exc)
    loaded_image = req.content
    loaded_image = cv2.imdecode(np.asarray(bytearray(loaded_image)), 1)
    loaded_image = cv2.cvtColor(loaded_image, cv2.COLOR_BGR2GRAY)
    return loaded_image

In [6]:
# Determine if input is link or path & return image
def get_image(which):
    """Decide if parameter is a link or directory, and load image from it.

    :param which: link or directory to where image is located
    :return: image pulled from link or directory
    """
    try:
        image = get_image_link(which)
        return image
    except:
        try:
            image, label = get_image_path(which)
            return image
        except:
            print('Load error. Try again')

In [7]:
# Blur image
def blur_image(original_image):
    """Blur image using median filter.

    :param original_image: original image to be blurred
    :return: blurred image
    """
    return cv2.medianBlur(original_image, 3)

In [8]:
# Closing (Dilation -> Erosion)
def morph_image(original_image):
    """Close(dilate and erode) given image to erase occluding lines.

    :param original_image: original image to be closed
    :return: closed image
    """
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 4))
    mor_img = cv2.morphologyEx(original_image, cv2.MORPH_CLOSE, kernel)
    return mor_img

In [9]:
# Image Cropping
def crop_image(original_image):
    """Crop margin out of given images.

    :param original_image: original image to be cropped
    :return: cropped image
    """
    cut = 5     # Boundary of image that is definitely not text
    x_list, y_list = [], []
    corners = cv2.goodFeaturesToTrack(original_image, 200, 0.01, 6)
    corners = np.int0(corners)
    for i in corners:
        x, y = i.ravel()
        if x > cut and y > cut:
            x_list.append(x)
            y_list.append(y)
    xl = min(x_list)
    xr = max(x_list)
    yt = min(y_list)
    yb = max(y_list)

    cr_img = cv2.copyMakeBorder(original_image[yt:yb, xl:xr],
                                5, 5, 5, 5, cv2.BORDER_CONSTANT, value=255)
    return cr_img

In [10]:
# Binarisation & Noise Removal
def bw(original_image):
    """Binarise given image to make it black & white.

    :param original_image: original image to be binarised
    :return: binarised image
    """
    t1 = original_image[0][0]
    t2 = original_image[0][original_image.shape[1]-1]
    t3 = original_image[original_image.shape[0]-1][0]
    t4 = original_image[original_image.shape[0]-1][original_image.shape[1]-1]
    t = min(t1, t2, t3, t4)
    if t > 250:
        bw_img = cv2.adaptiveThreshold(original_image, 255,
                                       cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY, 15, 2)
    else:
        thresh, bw_img = cv2.threshold(original_image, t, 255,
                                       cv2.THRESH_BINARY)
    return bw_img

In [11]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver import ActionChains
import unittest, time
import sys
import pandas as pd
import numpy as np 
import logbook, csv
from lxml import html
import os
# import Preprocessing as ps

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException

In [12]:
# step -1)  need web driver
#The WebDriver language APIs provides ways to pass capabilities to ChromeDriver.
#The exact mechanism differs by the language, but most languages use one or both of the following mechanisms:

#step-2) now from search engine application migrate to our required url using driver object
# step-3) driver.execute script
# step-4) driver.find_element by xpath where id we need to send id as keys if input is required otherwise like if only captcha required thn no key is to ge given
# 




In [14]:
service = Service('C:/Program Files (x86)/Google/Chrome/Application/chrome.exe')
options = webdriver.ChromeOptions()#1
driver = webdriver.Firefox()#service=service, options=options)

driver.get('https://mysprsemak.spr.gov.my/semakan/daftarPemilih') #2

There was an error managing geckodriver (Access is denied. (os error 5)); using driver found in the cache
There was an error managing firefox; using browser found in the cache


In [15]:
driver.execute_script("document.body.style.zoom='500%'")  

In [16]:
driver.find_element(By.XPATH,"//*[@name='NoKp']").send_keys('000201101765')

In [ ]:
captcha_element = driver.find_element(By.XPATH,"//*[@id='img_captcha']/img")

In [13]:
# os.chdir('C:/Users/00144791/Automation/TMNS/SAMPLE')
img = captcha_element.screenshot('C:\\Users\\MBBLABS\\Documents\\Web Scrapping\\captcha2.png')
# loaded_image = cv2.imdecode(np.asarray(bytearray(img)), 1)
# loaded_image = cv2.cvtColor(loaded_image, cv2.COLOR_BGR2GRAY)
# f = open("filename.png", 'wb')
# # data being whatever data you wanted to write
# f.write(loaded_image)
#img = Image.open(StringIO(img))
myimage=get_image('C:\\Users\\MBBLABS\\Documents\\Web Scrapping\\captcha2.png')
# window_name = 'Image'
# cv2.imshow(window_name, myimage)
# cv2.waitKey(0)
# window_name = 'Image'
# cv2.imshow(window_name, myimage)
# # cv2.imshow('Window',mimg)
# cv2.waitKey(0)
#img = fox.get_screenshot_as_png() and then img = Image.open(StringIO(img))
crimg=crop_image(myimage)
bwimg=bw(myimage)
mimg=morph_image(myimage)
median = cv2.medianBlur(myimage, 5)
#window_name = 'Image'
#cv2.imshow(window_name, myimage)
# cv2.imshow('Window',median)
# cv2.waitKey(0)
pytesseract.pytesseract.tesseract_cmd =r'C:\Users\MBBLABS\Tesseract-OCR\tesseract'
#         r'C:/Users/MBBLABS/anaconda3/Lib/site-packages/pytesseract/pytesseract'
# txt=pytesseract.image_to_string(median,lang='eng',config="--psm 4")
# output=txt.replace(" ", "").strip()
txt=pytesseract.image_to_string(median,config='--psm 10 --oem 3 -c tessedit_char_whitelist=0123456789')#,lang='eng',config="-psm 4") 
#txt1=ps.tesseract(mimg)
output=txt.strip()
print(output)
# print(txt)

driver.find_element(By.XPATH,"//*[@name='captcha']").send_keys(output)
time.sleep(2)
driver.find_element(By.XPATH,"//*[@type='submit']").click()

log=0
while 4==4:
    try:
        driver.find_element(By.CLASS_NAME,"text-red-600")
        print('login failed')
    except NoSuchElementException:
        print('Login successful')
        log=1
        break
    if log==0:
        driver.find_element(By.XPATH,"//*[@name='NoKp']").send_keys('000201101765')
        captcha_element = driver.find_element(By.XPATH,"//*[@id='img_captcha']/img")
        # os.chdir('C:/Users/00144791/Automation/TMNS/SAMPLE')
        img = captcha_element.screenshot('C:\\Users\\MBBLABS\\Documents\\Web Scrapping\\captcha2.png')
        # loaded_image = cv2.imdecode(np.asarray(bytearray(img)), 1)
        # loaded_image = cv2.cvtColor(loaded_image, cv2.COLOR_BGR2GRAY)
        # f = open("filename.png", 'wb')
        # # data being whatever data you wanted to write
        # f.write(loaded_image)
        #img = Image.open(StringIO(img))
        myimage=get_image('C:\\Users\\MBBLABS\\Documents\\Web Scrapping\\captcha2.png')
#         window_name = 'Image'
#         cv2.imshow(window_name, myimage)
#         cv2.waitKey(0)
        #img = fox.get_screenshot_as_png() and then img = Image.open(StringIO(img))
        crimg=crop_image(myimage)
        bwimg=bw(myimage)
        mimg=morph_image(myimage)
        median = cv2.medianBlur(myimage, 5)
        #window_name = 'Image'
        #cv2.imshow(window_name, myimage)
        # cv2.imshow('Window',mimg)
        # cv2.waitKey(0)
        pytesseract.pytesseract.tesseract_cmd =r'C:\Users\MBBLABS\Tesseract-OCR\tesseract'
               # r'C:\Program Files\Tesseract-OCR\tesseract'
        # txt=pytesseract.image_to_string(median,lang='eng',config="--psm 4")
        # output=txt.replace(" ", "").strip()
        txt=pytesseract.image_to_string(median,config='--psm 10 --oem 3 -c tessedit_char_whitelist=0123456789')#,lang='eng',config="-psm 4") 
        #txt1=ps.tesseract(mimg)
        output=txt.strip()
        print(output)
        # print(txt)

        driver.find_element(By.XPATH,"//*[@name='captcha']").send_keys(output)
        time.sleep(2)
        driver.find_element(By.XPATH,"//*[@type='submit']").click()
print(log)
table1=driver.find_element(By.XPATH,"/html/body/div[1]/main/div/div[2]/div/div[1]/table").text
table2=driver.find_element(By.XPATH,"//*[@class='mx-4']/div[2]/table/tbody/tr/td[2]").text
text1=table1.split("\n")#.split(" ")
print(text1)
text2=table2.split("\n")#.split(" ")
print(text2)
#ninx=text1.index("Nama")
icno=text1[1]
icsplit=icno.split(" ")
print(icsplit[1:])

# for i in icsplit[1:]:
#     j=i+j
    
# print(j)
    
# driver.close()
#print(driver.find_element(By.XPATH,"/html/body/div[1]/main/div/div[2]/div/div[2]/table").text[1])

#driver.find_element(By.CLASS_NAME,"text-red-600")
# else:
#driver.page_source()

There was an error managing geckodriver (Access is denied. (os error 5)); using driver found in the cache
There was an error managing firefox; using browser found in the cache


420
login failed
6555
Login successful
1
['MAKLUMAT PERIBADI PEMILIH', 'Nama Penuh TAN KAH ONN', 'No. Kad Pengenalan 000201101765', 'Tarikh Lahir 1 FEBRUARI 2000', 'Jantina LELAKI']
['095 / 09 / 15 / 001 - BAGAN PASIR']
['Penuh', 'TAN', 'KAH', 'ONN']
